# Übung 09: Object Detection

##  Aufgabe 2: Support Vector Machine

In dieser Aufgabe wollen wir Support Vector Machines betrachten. Obwohl auch OpenCV eine Implementation von SVMs beinhaltet möchten wir diesmal die Python Implementation von scikit-learn verwenden. Dazu müsses Sie allenfalls noch das enstprechende Packet mittels `pip3 install sklearn` installieren.

In einer früheren Übung haben wir von unseren Photobox Bildern mit grünen Hintergrund die Hände vom Hintergrund mittels Farbsegemtnierung getrennt. Die Bereiche mussten wir dabei selber finden. In dieser Übung möchten wir eine SVM einsetzen um diese Klassifizierung zu machen.

Zuerst wollen wir die Bilder laden und darstellen.


In [2]:
# OpenCV needs to be included first
import cv2
import numpy as np
from skimage.feature import hog
from sklearn.metrics import classification_report,accuracy_score

import tensorflow_core
from tensorflow.keras.datasets import cifar10

# we will use the svm from sklearn
from sklearn import svm

# for displaying images in jupyter

import matplotlib as mpl
from matplotlib import pyplot as plt
%matplotlib inline
mpl.rcParams['figure.dpi']= 200

AttributeError: module 'tensorflow.python.eager.context' has no attribute 'default_execution_mode'

In [ ]:
image  = cv2.imread('images/hand_small.png')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
label = cv2.imread('images/label_small.png')
plt.subplot(1, 2, 1)
plt.imshow(image)
plt.subplot(1, 2, 2)
plt.imshow(label)
print(image.shape)
print(label.shape)

Als Features für die SVM wollen wir 2D Daten benutzen um sie besser darstellen zu können. Da die Segmentierung auf den Farben basieren soll, zum Beispiel Hue und Saturation. Wir müssen also das Bild in HSV umwandeln.

Die SVM braucht keine Bilddaten, sondern nur ein Array von Werten.

In [ ]:
hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
hs = hsv[:,:,0:2]
data = hs.reshape((-1,2))

In [ ]:
label_grey = label[:,:,0]
classes = label_grey.reshape(-1)
print(classes.shape)

Da wir Pixel für die Trainingsdaten verwenden, erhalten wir sehr viele Trainings Daten. Für die SVM wird das recht aufwendig, deshalben sollten wir erst mal die Anzahl Daten reduzieren in dem wir nur jedes n-te Pixel nehmen. Späten können wir n verändern um bessere Ergebnisse mit mehr Trainingsdaten zu erhalten.

In [ ]:
n = 1000
classes_reduced = classes[1::n]
data_reduced = data[1::n,:]
print(classes_reduced.shape)
print(data_reduced.shape)

Wir können die Features nun als Scatterplot darstellen. Dazu möchten wir die Daten zuerst noch in die positiven und negativen Beispiele unterteilen.


In [ ]:
positives = data_reduced[classes_reduced==0]
negatives = data_reduced[classes_reduced==255]
fig, ax = plt.subplots(figsize=(12,8))
ax.scatter(positives[:,0], positives[:,1], marker='x', label='Positive')
ax.scatter(negatives[:,0], negatives[:,1], marker='o', label='Negative')
ax.legend()

Wir erzeugen einen SVM Klassifier mittels svm.SVC. Der Parameter C bestimmt den Fehlerterm, ein grösserer Wert von C zwingt die SVM mehr Samples richtig zu klassifizieren (auf Kosten von einer glätteren Entscheidungsfunktion).

In [ ]:
classifier = svm.SVC(C=1.0, gamma='auto')

Die Funktion fit trainiert den Klassifikator.

In [ ]:
classifier.fit(data_reduced, classes_reduced)

In [ ]:
classifier.score(data,classes)

Laden Sie nun das 2. Bild und überprüfen Sie, welchen Score dass sie auf diesem Bild erreichen mit der bereits trainierten SVM.

Berechnen Sie dann die Klassifizierung, die der Classifier auf den Daten berechnet (mittels predict) und stellen Sie das Resultat wieder als Bild dar (mittels reshape können Sie das 1D Resultat wieder in ein 2D Bild umwandeln)

## Aufgabe 3: SVM für Bild Klassifizierung

In dieser Aufgabe wollen wir nun statt einzelne Pixel ganze Bilder Klassifizieren. Dazu verwenden wir den CIFAR-10 Datensatz. Dieser enthält 50000 Bildern der Grösse 32x32x3 von 10 verschiedenen Klassen.

In [ ]:
(x_image_train, y_train), (x_image_test, y_test) = cifar10.load_data()

Wir möchten zuerst einige Bilder zufällig darstellen.

In [ ]:
label_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
def show_images(images, labels):
    fig, axes = plt.subplots(3, 5)
    fig.subplots_adjust(hspace=0.6, wspace=0.3)
    
    for i, ax in enumerate(axes.flat):
        # Plot image.
        example = np.random.randint(len(images))
        ax.imshow(images[example])
        
        # Name of the true class.
        cls_true_name = label_names[labels[example, 0]]
        xlabel = "{0}".format(cls_true_name)
        
        # Show the classes as the label on the x-axis.
        ax.set_xlabel(xlabel)
        
        # Remove ticks from the plot.
        ax.set_xticks([])
        ax.set_yticks([])
    return

In [ ]:
mpl.rcParams['figure.dpi']= 100
show_images(x_image_train, y_train)

Für die SVM brauchen wir Features, die aus den Bildern berechnet werden. Dazu möchten wir HOG Features verwenden. Schauen wir uns diese Features auf einem Beispiel Bild an:

In [ ]:
example_hog = hog(x_image_train[0], multichannel=True)
print(example_hog.shape)
print(example_hog)

Für SVM sind 50000 Bilder etwas viel, deshalb wollen wir für den Anfang weniger verwenden und den Anteil erhöhen. Dazu samplen wir 10000 Indizes.

In [ ]:
indices = np.random.choice(x_image_train.shape[0], 10000, replace=False)
print(max(indices))

In [ ]:
x_image_train_small = x_image_train[indices, :, :, :]
y_train_small = y_train[indices, :]
print(x_image_train_small.shape)
print(y_train_small.shape)

Von diesen Bildern sollen nun die Hog Features berechnet werden.

In [ ]:
def calculate_features_hog(x):
    feature_vec = []
    for i in range(x.shape[0]):
        f = hog(x[i], multichannel=True)
        feature_vec.append(f)
    return np.asarray(feature_vec)

Berechnen sie nun die Features auf den 5000 Hog Features... 

 und trainieren sie eine SVM auf diesen Features:

Um das Resultat auszuwerten wollen wir die confusion matrix berechnen und anzeigen. Dazu müssen wir 
mit classifier.predict zuerst die Prediction berechnen. Anschliessend kann die Confusion Matrix mittels
 sklearn.metrics.confusion_matrix berechnet werden.

Die Funktion sklearn.metrics.classification_report zeigt eine Übersicht über das Ergebnis:

Natürlich sollten wir das auf dem Test Set und nicht auf dem Trainings Set auswerten. Wie sehen die Resultate da aus?